In [5]:
import torch
from transformers import AutoModel, AutoTokenizer 

bertweet = AutoModel.from_pretrained("vinai/bertweet-covid19-base-uncased")

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-covid19-base-uncased")

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
import tweepy as tw
import numpy as np
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
import gensim
from sklearn.feature_extraction.text import CountVectorizer

In [1]:
def cleaner2(x):
    import re
    x = re.sub(r'https:\/\/.*[\r\n]*', '', x)
    x = re.sub(',', '',x)
    x = re.sub('\?','',x)
    x = re.sub('!','',x)
    x = re.sub('\n', ' ',x)
    x = re.sub("@[A-Za-z0-9]+",'', x)
    x = x.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    
    #from gensim.parsing.preprocessing import remove_stopwords
    #x = remove_stopwords(x)
    
    import emoji
    cleaned = emoji.get_emoji_regexp().sub("", x)

    return cleaned



In [ ]:
file_path = "/Users/wangxinyu/Downloads/Hate-Narratives-main/hydrated.csv"
colnames = ['tweet_id', 'text', 'num_likes', 'date', 'author']
hydrated_data = pd.read_csv(file_path, names = colnames)
hydrated_data.info()
hydrated_data.head()

In [ ]:
clean_bertcorpus=hydrated_data.apply(lambda x: cleaner2(x['text']), axis=1)
clean_bertcorpus2=clean_bertcorpus.tolist()


In [ ]:
print(len(clean_bertcorpus2))
print(len(text))

In [ ]:
tweet_ids=hydrated_data['tweet_id'].tolist()
text=hydrated_data['text'].tolist()
text[0]

In [ ]:

features=[]
for item in clean_bertcorpus2:
    print(item)
    input_ids = torch.tensor([tokenizer.encode(item)])
    print(np.shape(input_ids))

    with torch.no_grad():
        feature=bertweet(input_ids).last_hidden_state[0,0,:].tolist()
        print(np.shape(feature))
        features.append(feature)
        


In [ ]:
print(np.shape(feature))

In [ ]:
from bertopic import BERTopic

In [ ]:
from sklearn.feature_extraction import text 

additional_stop_words = ["ur", '21', 'amb', 'blah', 'till', 'yep', 'rt', 'yall', '23', '19', 'guy', 'hes', '14',
                        'okay', 'whats', 'huh', 'jan', 'did', 'didnt', 'mean', 'does', 'youre', 'yeah', 'hey', 'just',
                        'like', 'yes', 'yessss', 'yesssss', 'plz', '40', 'agree', 'zy', 'zvinopfuura', 'yess',
                         'zzzzzzzz', 'zzzzzzzz', 'yessssss', 'yesss', 'gt', 'gtgtgt', 'etcwith', 'etchowever', 'etcwhy',
                        'etcthere', 'etcthey', 'etcpoc', 'zzzzzz', 'etcit', 'etcthe', 'etcon', 'etcl',
                        'fuck', 'etcetera', 'etcetc','im']
s_words = text.ENGLISH_STOP_WORDS.union(additional_stop_words)

from sklearn.feature_extraction.text import CountVectorizer
vectorizer_custom = CountVectorizer(max_df = 0.70, min_df = 0.05, stop_words = s_words)

from hdbscan import HDBSCAN

#HDBSCAN
hdbscan_custom = HDBSCAN(min_cluster_size=15, metric='euclidean', 
                        cluster_selection_method='leaf', prediction_data = True, cluster_selection_epsilon = 1.0)

In [ ]:
alberto = BERTopic(language='english', 
                   calculate_probabilities=True, 
                   verbose=True, 
                   vectorizer_model=vectorizer_custom,
                   hdbscan_model= hdbscan_custom
                  )

In [ ]:
features_array=np.array(features)
features_array.shape

In [ ]:
topics, probabilities = alberto.fit_transform(clean_bertcorpus2, features_array)

In [ ]:
freq = alberto.get_topic_info()
freq
#alberto.get_topic(1)

In [ ]:
alberto.save('bertopic_model')

In [ ]:
og_data = pd.read_csv("all_tweet_ids.csv")

labels = (og_data.loc[(og_data.Label == 'Hate') | (og_data.Label == 'Counterhate')]\
                     .rename(columns = {"Tweet ID" : "tweet_id"})
             )

#join label information to hydrated data
hydrated_data1 = pd.merge(hydrated_data, labels, how = 'left', on = 'tweet_id')

In [ ]:
count_hate=0
count_Counterhate=0
Else=0
hydrated_data2=hydrated_data1[0:1000]
for i in hydrated_data2['Label']:
    if i=='Counterhate':
        count_Counterhate+=1
    elif i=='Hate':
        count_hate+=1
    else:
        Else+=1


In [ ]:
print(count_hate,count_Counterhate,Else)

In [ ]:
hydrated_data1.head(50)

In [2]:
import matplotlib.pyplot as plt
import itertools

colors = itertools.cycle(["r", "b", "g"])
import umap
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=768, 
                            metric='cosine').fit_transform(features_array)
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

# Prepare data
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(features_array)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
#plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(outliers.x, outliers.y, color=next(colors), s=0.1)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.1, cmap='hsv_r')
plt.colorbar()


NameError: name 'features_array' is not defined

In [ ]:
alberto.visualize_distribution(probabilities[0])

In [ ]:
alberto.visualize_topics()